In [17]:
import pandas as pd
import numpy as np
from itertools import product
import math
import random


According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [150]:
class IsingModel:
    def __init__(self, data, lr=0.01):
        self.lr = lr
        
        self.activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        self.spins_correlation = self.activations_matrix.corr().values
        lind = np.tril_indices_from(self.spins_correlation)
        self.spins_correlation[lind]=0
        self.spins_average = self.activations_matrix.mean().values

        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=.01, size=self.spins_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        self.H = self.activations_matrix.sum()/self.activations_matrix.shape[0]
        
    def train(self):
        np.random.seed(3)
        
        totalParamVariation = math.inf
        stopCondition = 0.5
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < 1000:
            # Calculate P(sigma) for every possible combination in the model
            spin_values = [-1,1]
            prob_dict = {}
            for sigma in product(spin_values, repeat=self.activations_matrix.shape[1]):
                prob_dict[sigma] = 0
                for i in range(0, self.activations_matrix.shape[1]):
                    for j in range(i+1, self.activations_matrix.shape[1]):
                        prob_dict[sigma] += sigma[i]*sigma[j]*self.J[i][j]

                for i in range(0,self.activations_matrix.shape[1]):
                    prob_dict[sigma] += sigma[i]*self.H[i]

                prob_dict[sigma] = math.exp(prob_dict[sigma])

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            # Calculate the expectation of the correlations
            corr_model_exp = np.zeros(self.J.shape)
            for sigma in product(spin_values, repeat=self.activations_matrix.shape[1]):
                for i in range(0, self.activations_matrix.shape[1]):
                    for j in range(i+1, self.activations_matrix.shape[1]):
                        corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

            # Calculate the expectation of the averages
            mean_model_exp = np.zeros(self.H.shape)
            for sigma in product(spin_values, repeat=self.activations_matrix.shape[1]):
                for i in range(0, self.activations_matrix.shape[1]):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.spins_correlation - corr_model_exp)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(self.J - stepJ).sum().sum()
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = 0
            totalParamVariation = deltaH + deltaJ
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch += 1

        print(self.H)
        print(spins_correlation)
        print(self.J)
        
    def trainGlauber(self, data, lr=0.1):
        #Generate a random initial spin configuration
        state = random.choices([-1,1], k=self.activations_matrix.shape[1])
        spin_to_fip = random.choice(range(0,self.activations_matrix.shape[1]))

        print('')

In [121]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]

In [122]:
ising = IsingModel(activations_list)
ising.train()

Epoch 1 TotalParamVariation 1.0379739765157274
Epoch 2 TotalParamVariation 1.0399572386450247
Epoch 3 TotalParamVariation 1.0419121724080067
Epoch 4 TotalParamVariation 1.0515246500228563
Epoch 5 TotalParamVariation 1.0614377352282531
Epoch 6 TotalParamVariation 1.071224321446509
Epoch 7 TotalParamVariation 1.0808866869714546
Epoch 8 TotalParamVariation 1.090427070144603
Epoch 9 TotalParamVariation 1.0998476698442552
Epoch 10 TotalParamVariation 1.1091506459817713
Epoch 11 TotalParamVariation 1.1183381200039626
Epoch 12 TotalParamVariation 1.1274121754006425
Epoch 13 TotalParamVariation 1.1363748582164266
Epoch 14 TotalParamVariation 1.1452281775659316
Epoch 15 TotalParamVariation 1.1539741061515878
Epoch 16 TotalParamVariation 1.1626145807833193
Epoch 17 TotalParamVariation 1.171151502899405
Epoch 18 TotalParamVariation 1.1795867390878794
Epoch 19 TotalParamVariation 1.1879221216078748
Epoch 20 TotalParamVariation 1.196159448910353
Epoch 21 TotalParamVariation 1.204300486157707
Epoch 

Epoch 320 TotalParamVariation 2.2292586007512236
Epoch 321 TotalParamVariation 2.231112353025625
Epoch 322 TotalParamVariation 2.2329616680776194
Epoch 323 TotalParamVariation 2.234806564743342
Epoch 324 TotalParamVariation 2.2366470617330076
Epoch 325 TotalParamVariation 2.2384831776323657
Epoch 326 TotalParamVariation 2.2403149309041375
Epoch 327 TotalParamVariation 2.242142339889422
Epoch 328 TotalParamVariation 2.243965422809075
Epoch 329 TotalParamVariation 2.2457841977650705
Epoch 330 TotalParamVariation 2.2475986827418275
Epoch 331 TotalParamVariation 2.2494088956075244
Epoch 332 TotalParamVariation 2.2512148541153807
Epoch 333 TotalParamVariation 2.253016575904919
Epoch 334 TotalParamVariation 2.254814078503208
Epoch 335 TotalParamVariation 2.25660737932608
Epoch 336 TotalParamVariation 2.2583964956793268
Epoch 337 TotalParamVariation 2.260181444759876
Epoch 338 TotalParamVariation 2.2619622436569484
Epoch 339 TotalParamVariation 2.2637389093531946
Epoch 340 TotalParamVariation

Epoch 651 TotalParamVariation 2.6757792094304715
Epoch 652 TotalParamVariation 2.6767763232734016
Epoch 653 TotalParamVariation 2.6777719599563197
Epoch 654 TotalParamVariation 2.6787661234092264
Epoch 655 TotalParamVariation 2.6797588175470017
Epoch 656 TotalParamVariation 2.6807500462694804
Epoch 657 TotalParamVariation 2.6817398134615225
Epoch 658 TotalParamVariation 2.6827281229930877
Epoch 659 TotalParamVariation 2.6837149787193053
Epoch 660 TotalParamVariation 2.684700384480549
Epoch 661 TotalParamVariation 2.6856843441025067
Epoch 662 TotalParamVariation 2.686666861396252
Epoch 663 TotalParamVariation 2.6876479401583153
Epoch 664 TotalParamVariation 2.6886275841707508
Epoch 665 TotalParamVariation 2.6896057972012124
Epoch 666 TotalParamVariation 2.6905825830030183
Epoch 667 TotalParamVariation 2.691557945315222
Epoch 668 TotalParamVariation 2.6925318878626796
Epoch 669 TotalParamVariation 2.6935044143561204
Epoch 670 TotalParamVariation 2.6944755284922146
Epoch 671 TotalParamVar

Epoch 960 TotalParamVariation 2.928396623249655
Epoch 961 TotalParamVariation 2.9290709494957867
Epoch 962 TotalParamVariation 2.929744540628161
Epoch 963 TotalParamVariation 2.930417398059762
Epoch 964 TotalParamVariation 2.9310895231995717
Epoch 965 TotalParamVariation 2.9317609174525874
Epoch 966 TotalParamVariation 2.9324315822198397
Epoch 967 TotalParamVariation 2.9331015188984
Epoch 968 TotalParamVariation 2.933770728881405
Epoch 969 TotalParamVariation 2.934439213558063
Epoch 970 TotalParamVariation 2.9351069743136713
Epoch 971 TotalParamVariation 2.9357740125296328
Epoch 972 TotalParamVariation 2.9364403295834682
Epoch 973 TotalParamVariation 2.9371059268488318
Epoch 974 TotalParamVariation 2.9377708056955223
Epoch 975 TotalParamVariation 2.9384349674895045
Epoch 976 TotalParamVariation 2.9390984135929146
Epoch 977 TotalParamVariation 2.9397611453640797
Epoch 978 TotalParamVariation 2.9404231641575334
Epoch 979 TotalParamVariation 2.9410844713240234
Epoch 980 TotalParamVariatio

NameError: name 'spins_correlation' is not defined

In [113]:
random.choices([-1,1], k=4)


[1, 1, 1, -1]

In [149]:
random.choice(range(0,4))

0